In [5]:
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras import optimizers
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
def armazenar_nomes_arquivos(caminho):
    nomes_arquivos = []
    for filename in os.listdir(caminho):
        filepath = os.path.join(caminho, filename)
        nomes_arquivos.append(filepath)

    return nomes_arquivos

def criar_df_classe_unica(nomes_arquivos, label):
    labels = []
    for i in range(len(nomes_arquivos)):
        labels.append(label)

    df_classe_unica = pd.DataFrame({'nomes_arquivos': nomes_arquivos, 'labels': labels})

    return df_classe_unica

def criar_modelo():
    model = Sequential() 
    model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(300, 300, 3))) 
    model.add(MaxPooling2D(pool_size=(3,3))) 
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Conv2D(256, kernel_size=(3, 3)))
    model.add(Flatten()) 
    model.add(Dense(200, activation='relu')) 
    model.add(Dense(1, activation='sigmoid')) 
    
    return model

def rotacionar_imagem(imagem):
    imagem_cinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)

    _, imagem_bin = cv2.threshold(imagem_cinza, 127, 255, cv2.THRESH_BINARY)

    contornos, _ = cv2.findContours(imagem_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    maior_contorno = max(contornos, key=cv2.contourArea)
    
    rect = cv2.minAreaRect(maior_contorno)
    box = cv2.boxPoints(rect)
    box = np.intp(box)

    _, dimensoes, angulo = rect
    
    largura, altura = dimensoes
    if largura < altura:
        angulo = angulo + 90 

    altura, largura = imagem.shape[:2]
    centro = (largura // 2, altura // 2)
    matriz_rotacao = cv2.getRotationMatrix2D(centro, angulo, 1.0)
    imagem_rotacionada = cv2.warpAffine(imagem, matriz_rotacao, (largura, altura))
    
    return imagem_rotacionada

# Importa Imagens

In [8]:
kirmizi_path = '/kaggle/input/pistachio-image-dataset/Pistachio_Image_Dataset/Pistachio_Image_Dataset/Kirmizi_Pistachio'
siirt_path = '/kaggle/input/pistachio-image-dataset/Pistachio_Image_Dataset/Pistachio_Image_Dataset/Siirt_Pistachio'

kirmizi_arquivos = armazenar_nomes_arquivos(kirmizi_path)
df_kirmizi = criar_df_classe_unica(kirmizi_arquivos, 'kirmizi')

siirt_arquivos = armazenar_nomes_arquivos(siirt_path)
df_siirt = criar_df_classe_unica(siirt_arquivos, 'siirt')

df = pd.concat([df_kirmizi, df_siirt], axis=0, ignore_index=True)


##Label = 1 -> Kirmizi e label = 0 -> Siirt
df['labels'] = df['labels'].map({'kirmizi': 1, 'siirt': 0})

X = df['nomes_arquivos']
y = df['labels']


# Redimensiona e Alinha Imagens

In [9]:
imagens_rotacionadas = []
imagens_resize = []

for arquivo in X:
    imagem = cv2.imread(arquivo)
    nl, nc, num_canais = imagem.shape
    imagem_resize = cv2.resize(imagem, None, fx=0.5, fy=0.5)
    imagem_rotacionada = rotacionar_imagem(imagem_resize)
    imagens_resize.append(imagem_resize)
    imagens_rotacionadas.append(imagem_rotacionada)

imagens_rotacionadas = np.array(imagens_rotacionadas)
imagens_resize = np.array(imagens_resize)

imagens_rotacionadas = np.float32(imagens_rotacionadas/255)
imagens_resize = np.float32(imagens_resize/255)

X_resize_treino, X_resize_teste, y_resize_treino, y_resize_teste = train_test_split(imagens_resize, y, test_size=0.5, stratify=y, random_state=42)

X_alinha_treino, X_alinha_teste, y_alinha_treino, y_alinha_teste = train_test_split(imagens_rotacionadas, y, test_size=0.5, stratify=y, random_state=42)

print(X_resize_treino.shape)
print(X_resize_teste.shape)

print(X_alinha_treino.shape)
print(X_alinha_teste.shape)


(1074, 300, 300, 3)
(1074, 300, 300, 3)
(1074, 300, 300, 3)
(1074, 300, 300, 3)


# Treinamento de modelo com imagens resize

In [12]:
model = criar_modelo()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_resize_treino, y_resize_treino, batch_size=100, epochs=20, verbose=2)

score = model.evaluate(X_resize_teste, y_resize_teste, verbose=False)
print('Test loss: %.4f'%(score[0]))
print('Test accuracy: %.2f %%'%(100*score[1]))
print('Test error: %.2f %%'%(100*(1-score[1])))

Epoch 1/20
11/11 - 52s - 5s/step - accuracy: 0.5885 - loss: 0.9665
Epoch 2/20
11/11 - 49s - 4s/step - accuracy: 0.7402 - loss: 0.5218
Epoch 3/20
11/11 - 50s - 5s/step - accuracy: 0.7980 - loss: 0.4176
Epoch 4/20
11/11 - 49s - 4s/step - accuracy: 0.7952 - loss: 0.4105
Epoch 5/20
11/11 - 49s - 4s/step - accuracy: 0.8175 - loss: 0.4040
Epoch 6/20
11/11 - 49s - 4s/step - accuracy: 0.8296 - loss: 0.3669
Epoch 7/20
11/11 - 49s - 4s/step - accuracy: 0.8529 - loss: 0.3404
Epoch 8/20
11/11 - 82s - 7s/step - accuracy: 0.8650 - loss: 0.3041
Epoch 9/20
11/11 - 60s - 5s/step - accuracy: 0.8855 - loss: 0.2804
Epoch 10/20
11/11 - 59s - 5s/step - accuracy: 0.8510 - loss: 0.3225
Epoch 11/20
11/11 - 49s - 4s/step - accuracy: 0.8510 - loss: 0.3326
Epoch 12/20
11/11 - 50s - 5s/step - accuracy: 0.8873 - loss: 0.2657
Epoch 13/20
11/11 - 49s - 4s/step - accuracy: 0.8836 - loss: 0.2579
Epoch 14/20
11/11 - 50s - 5s/step - accuracy: 0.9115 - loss: 0.2155
Epoch 15/20
11/11 - 49s - 4s/step - accuracy: 0.9236 - lo

In [11]:
model = criar_modelo()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_alinha_treino, y_alinha_treino, batch_size=100, epochs=20, verbose=2)

score = model.evaluate(X_alinha_teste, y_alinha_teste, verbose=False)
print('Test loss: %.4f'%(score[0]))
print('Test accuracy: %.2f %%'%(100*score[1]))
print('Test error: %.2f %%'%(100*(1-score[1])))

Epoch 1/20
11/11 - 51s - 5s/step - accuracy: 0.6266 - loss: 0.8105
Epoch 2/20
11/11 - 49s - 4s/step - accuracy: 0.7877 - loss: 0.4469
Epoch 3/20
11/11 - 49s - 4s/step - accuracy: 0.8007 - loss: 0.4026
Epoch 4/20
11/11 - 48s - 4s/step - accuracy: 0.8352 - loss: 0.3578
Epoch 5/20
11/11 - 49s - 4s/step - accuracy: 0.8277 - loss: 0.3707
Epoch 6/20
11/11 - 49s - 4s/step - accuracy: 0.8566 - loss: 0.3293
Epoch 7/20
11/11 - 49s - 4s/step - accuracy: 0.8790 - loss: 0.2837
Epoch 8/20
11/11 - 49s - 4s/step - accuracy: 0.8473 - loss: 0.3241
Epoch 9/20
11/11 - 49s - 4s/step - accuracy: 0.8501 - loss: 0.3199
Epoch 10/20
11/11 - 49s - 4s/step - accuracy: 0.8557 - loss: 0.3145
Epoch 11/20
11/11 - 49s - 4s/step - accuracy: 0.8994 - loss: 0.2426
Epoch 12/20
11/11 - 49s - 4s/step - accuracy: 0.8994 - loss: 0.2362
Epoch 13/20
11/11 - 49s - 4s/step - accuracy: 0.9218 - loss: 0.1958
Epoch 14/20
11/11 - 49s - 4s/step - accuracy: 0.9320 - loss: 0.1593
Epoch 15/20
11/11 - 49s - 4s/step - accuracy: 0.9367 - lo